## Домашка 4

In [2]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.ensemble import RandomForestClassifier
import gensim
import csv
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import pickle
import numpy as np

In [13]:
import adagram

In [10]:
train, test = pd.read_csv('train_6b.csv'), pd.read_csv('test_6b.csv')
# с диплома остался парафразер с очень тщательной нормализацией
print(train.head())
print(train.shape)
print(test.head())
print(test.shape)

   Unnamed: 0                                      предложение 1  \
0           0  Полицейским разрешат стрелять на поражение по ...   
1           1  Право полицейских на проникновение в жилище ре...   
2           2  Президент Египта ввел чрезвычайное положение в...   
3           3  Вернувшихся из Сирии россиян волнует вопрос тр...   
4           4  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                       предложение 2  класс  \
0  Полиции могут разрешить стрелять по хулиганам ...      0   
1  Правила внесудебного проникновения полицейских...      0   
2  Власти Египта угрожают ввести в стране чрезвыч...      0   
3  Самолеты МЧС вывезут россиян из разрушенной Си...     -1   
4  Самолеты МЧС вывезут россиян из разрушенной Си...      0   

                                             леммы 1  \
0  полицейский разрешать стрелять поражение гражд...   
1  право полицейский проникновение жилище решать ...   
2  президент египет вводить чрезвычайный пол

In [3]:
# чтение одного мистического корпуса
sentences = []
sentences_raw = []
for file in os.listdir(os.path.join(os.getcwd(),'corpus')):
    with open('corpus/' + file, 'r', encoding='utf-8') as  f:
        sentence = []
        reader = csv.reader(f, delimiter=';')
        for line in reader:
            word = line[2]
            if word.isalpha():
                sentence.append(word.lower())
        sentences.append(sentence)
        sentences_raw.append(' '.join(sentence))

In [30]:
print(sentences[0][:20])

['год', 'на', 'ямал', 'более', 'или', 'менее', 'нормальный', 'погода', 'держаться', 'в', 'сочи', 'несколько', 'день', 'начинать', 'портиться', 'связанный', 'это', 'с', 'прохождение', 'атмосферный']


In [4]:
cv = CountVectorizer( max_features=1000)
X = cv.fit_transform(sentences_raw)

In [5]:
svd = TruncatedSVD(50)
svd.fit(X)
pickle.dump(svd, open('svd', 'wb'))
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}
id2vec_svd = svd.components_.T
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar
print(most_similar('погода', id2vec_svd))

['погода', 'мчс', 'снежный', 'пожарный', 'столица', 'госпитализировать', 'температура', 'чп', 'сообщение', 'встречный']


In [15]:
nmf = NMF(6)
nmf.fit(X)
pickle.dump(nmf, open('nmf', 'wb'))
id2vec_nmf = nmf.components_.T
print(most_similar('погода', id2vec_nmf))

['погода', 'мчс', 'снежный', 'ожидаться', 'полоса', 'ночь', 'пожарный', 'секунда', 'трасса', 'аэропорт']


In [20]:
fast_text = gensim.models.FastText(sentences, min_n=1)
fast_text.save('fasttext')

In [21]:
fast_text.wv.most_similar('погода')

[('подмога', 0.9999983310699463),
 ('пахомов', 0.9999969005584717),
 ('пожаро', 0.9999966621398926),
 ('выгода', 0.9999966025352478),
 ('автодром', 0.999996542930603),
 ('автодорога', 0.9999964237213135),
 ('автопром', 0.9999961853027344),
 ('вода', 0.9999960064888),
 ('автодор', 0.9999960064888),
 ('ягода', 0.9999960064888)]

In [23]:
w2v = gensim.models.Word2Vec(sentences, size=50, sg=1)
w2v.save('w2v')

In [28]:
w2v.wv.most_similar('мчс')

[('областной', 0.9963815212249756),
 ('региональный', 0.9956607818603516),
 ('прокуратура', 0.9955354332923889),
 ('администрация', 0.9954781532287598),
 ('брянский', 0.9953436851501465),
 ('псковский', 0.9945963025093079),
 ('су', 0.993961751461029),
 ('правительство', 0.9921631813049316),
 ('край', 0.9914276003837585),
 ('информация', 0.991313636302948)]

In [ ]:
train = pd.concat([train,test]) # возьмем все, а потом сделаем к фолд
cv_train1 = cv.transform(train['леммы 1']
cv_train2 = cv.transform(train['леммы 2']                    
svd_feature = cosine_distances(svd.transform(cv_train1), svd.transform(cv_train2))
nmf_feature = cosine_distances(nmf.transform(cv_train1),nmf.transform(cv_train2))

In [ ]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [ ]:
dim = 50
X_text_1_w2v = np.zeros((len(train['леммы 1']), dim))
X_text_2_w2v = np.zeros((len(train['леммы 2']), dim))

for i, text in enumerate(train['леммы 1'].apply(split()).values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(len(train['леммы 2'].apply(split()).values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)
                         
w2v_feature = cosine_distances(X_text_1_w2v,X_text_2_w2v)

In [ ]:
dim = 50
X_text_1_fast_text = np.zeros((len(train['леммы 1']), dim))
X_text_2_fast_text = np.zeros((len(train['леммы 2']), dim))

for i, text in enumerate(train['леммы 1'].values):
    X_text_1_fast_text[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(len(train['леммы 2'].values):
    X_text_2_fast_text[i] = get_embedding(text, fast_text, dim)
                         
fast_text_feature = cosine_distances(X_text_1_fast_text,X_text_2_fast_text)

In [ ]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v # просто умножаем вектор на частоту
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
dim = 100
X_text_1_adagram = np.zeros((len(train['леммы 1']), dim))
X_text_2_adagram = np.zeros((len(train['леммы 1']), dim))

for i, text in enumerate(train['леммы 1'].values):
    X_text_1[i] = get_embedding_adagram(text, vm, 5, dim)
    
for i, text in enumerate(train['леммы 1'].values):
    X_text_2[i] = get_embedding_adagram(text, vm, 5, dim)
adagram_feature = cosine_distances(X_text_1_adagram,X_text_2_adagram)

In [ ]:
X = np.concatenate([adagram_feature,fast_text_feature,w2v_feature,svd_feature,nmf_feature])
y= train['класс']

In [ ]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

## Как улучшить

* вместо эмбеддингов брать метрики по RuWordNet (улучшится ненамного, и не хочется повторять диплом)
* разметить например UDPipe и считать схожесть между словами с одинаковыми синтаксическими тегами

нет ничего стремнее, чем использовать чужие норм идеи, когда нет своих

In [5]:
# вот отсюда https://github.com/ufal/udpipe/blob/master/bindings/python/examples/udpipe_model.py
import ufal.udpipe 

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [12]:
model = Model('russian-syntagrus-ud-2.0-170801.udpipe')
sent1, sent2 = [], []

def to_conllu(sent, out):
    for tr in sent:
        sentences = model.tokenize(tr)
        for s in sentences:
            model.tag(s)
            model.parse(s)
        conllu = model.write(sentences, "conllu")
        out.append(conllu)

to_conllu(train['предложение 1'], sent1)
to_conllu(train['предложение 2'], sent2)
train.loc[:,'syntax1']=pd.Series(sent1)
train.loc[:,'syntax2']=pd.Series(sent2)
train.to_csv('paraphraser_train.csv') # парсинг работает быстро, но не совсем, поэтому сохраним
sent1, sent2 = [], []
to_conllu(test['предложение 1'], sent1)
to_conllu(test['предложение 2'], sent2)
test.loc[:,'syntax1']=pd.Series(sent1)
test.loc[:,'syntax2']=pd.Series(sent2)
test.to_csv('paraphraser_test.csv')
print(train.head())

   Unnamed: 0                                      предложение 1  \
0           0  Полицейским разрешат стрелять на поражение по ...   
1           1  Право полицейских на проникновение в жилище ре...   
2           2  Президент Египта ввел чрезвычайное положение в...   
3           3  Вернувшихся из Сирии россиян волнует вопрос тр...   
4           4  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                       предложение 2  класс  \
0  Полиции могут разрешить стрелять по хулиганам ...      0   
1  Правила внесудебного проникновения полицейских...      0   
2  Власти Египта угрожают ввести в стране чрезвыч...      0   
3  Самолеты МЧС вывезут россиян из разрушенной Си...     -1   
4  Самолеты МЧС вывезут россиян из разрушенной Си...      0   

                                             леммы 1  \
0  полицейский разрешать стрелять поражение гражд...   
1  право полицейский проникновение жилище решать ...   
2  президент египет вводить чрезвычайный пол